### 不同路径<br>
![微信截图_20191007195853.png](https://i.loli.net/2019/10/08/O2xylGmpJMbCTXt.png)
![微信截图_20191007195908.png](https://i.loli.net/2019/10/08/M6gTlXZqdmnEVQ2.png)


$dp[i][j] = dp[i-1][j] + dp[i][j-1]$if:  i > 0 and j > 0

$dp[i][j] = dp[i-1][j]$ if j = 0

$dp[i][j] = dp[i][j-1]$ if i = 0

自己写的，通过了，贼他妈开心！dp网格法，走到终点的路径可以拆分成走到邻居的路径之和。规则是任何一个点都可以拆分成从左边到或者是从下边到，不能反过来。当处于最边缘时x=0 or y=0，最佳路径只有一条(base情况)。

In [1]:
class Solution(object):
    def uniquePaths(self, m, n):
        dp = [[0]*m for _ in range(n)]
        for y in range(n):
            for x in range(m):
                if x > 0 and y > 0:
                    dp[y][x] = dp[y][x-1] + dp[y-1][x]
                elif x == 0:
                    dp[y][x] = 1
                elif y == 0:
                    dp[y][x] = 1
        return dp[n-1][m-1]

- 我们可以发现更新每一个值的时候，除了x=0和y=0那一行一列，其余值都依赖于前一个值和前一行对应下标的值
- 据此我们可以优化一下空间复杂度
- 每到新的一行，就等于前一行+前一个

In [2]:
class Solution:
    def uniquePaths(self, m: int, n: int) -> int:
        cur = [1] * n
        for i in range(1, m):
            for j in range(1, n):
                cur[j] += cur[j-1]
        return cur[-1]

### 零钱兑换<br>
![微信截图_20191007203227.png](https://i.loli.net/2019/10/08/csMprwn6oaNXADS.png)

![微信截图_20191007212512.png](https://i.loli.net/2019/10/08/PHpIJmQKSeMGoch.png)

主要思想和上图差不多，关键是要把公式列出来！每个当前数额的找零数值等于，当前数额分别减去coins中每一个coin得到的f(i-c)的最小值加1。边界条件注意一下，i-c要大于等于0，f(i-c)也要大于等于0。若找不开零钱则返回-1.

In [15]:
class Solution(object):
    def coinChange(self, coins, amount):
        ans = [0] * (amount+1)
        i = 1
        while i <= amount:
            temp = [ans[i-c] for c in coins if i>=c and ans[i-c]>=0]
            ans[i] = min(temp)+1 if temp else -1
            i += 1
        return ans[-1]  

### 最长上升子序列<br>

![微信截图_20191008180943.png](https://i.loli.net/2019/10/09/lwOn6qmAr2VxKiM.png)

- 自己的做法，dp\[i]=max(dp[j])+1 when nums[j] < nums[i]

In [19]:
class Solution(object):
    def lengthOfLIS(self, nums):
        if not nums: return 0
        n = len(nums)
        dp = [1] * n
        for i in range(1,n):
            temp = []
            for j in range(i-1,-1,-1):
                if nums[j] < nums[i]:
                    temp.append(dp[j])
            dp[i] = max(temp) + 1 if temp else 1
        return max(dp)
#       简化写法，更为优雅
#         for i in range(1,n):
#             dp[i] = max([1] + [dp[j]+1 for j in range(i) if nums[j]<nums[i]])
#         return max(dp)

维护一个递增的子序列，遍历数组，将每一个n的值通过二分比较的方式放入合适的地方。最开始递增子序列不存在任何值，则把n放入子序列最后一位。用i,j标记左右两侧，res记录当前子序列的长度。对于以后的每一个n，若n的值很大，大到超过了子序列的最后一个值，显然二分法会使得i不停地变成m+1，最后不满足i<j的条件，然后把这个n的值放入递增子序列的最后一位。若n的值很小，则二分法会调整i和j的值，使得n出现在合适的位置。例如\[1,3,4]碰到了2，则会变成\[1,2,4]。

总结一下就是，小值在数组内原地调整替代，大值则往外前进一步，res时刻记录好长度。

In [25]:
class Solution(object):
    def lengthOfLIS(self, nums):
        tails, res = [0] * len(nums), 0
        for n in nums:
            i, j = 0, res
            while i < j:
                m = (i+j)//2
                if tails[m] < n: i = m + 1
                else: j = m
            tails[i] = n
            if j == res:
                res += 1
        return res

### 戳气球<br>
![微信截图_20191009113442.png](https://i.loli.net/2019/10/09/jIzQCfhGE9mMteu.png)

- dp[i][j] 表示戳破(i,j)号气球的最大金币数量,
- 对i,j,k有: i<k<j, k代表被戳的某个气球
- dp[2][5]表示戳破(2,5)间气球的最大收益,k可取3or4,对比先戳3的和先戳4的收益
- dp[i][j] = max {for k = range(i+1, j -1) nums[i] * nums[k] * nums[j] + dp[i][k] + dp[k][j]}

![微信截图_20191009134714.png](https://i.loli.net/2019/10/10/kOzs6QBfT8YZtSj.png)

In [ ]:
class Solution:
    def maxCoins(self, nums: List[int]) -> int:
        
        nums = [1] + nums + [1]          # 添加左右二乘子
        n = len(nums)
        dp = [[0]*n for _ in range(n)]   # 创建dp二维数组方便后续计算
        
        for i in range(n-1,-1,-1):       # 从后往前遍历i:5->4->3->...
            for j in range(i+2,n):       # j须满足j>=i+2,这样才有k的空间
                for k in range(i+1,j):   # k满足:dp[2][4],k=3或dp[2][5],k=3or4
                    dp[i][j] = max(dp[i][j], nums[i]*nums[k]*nums[j]+dp[i][k]+dp[k][j])
        
        return dp[0][n-1]

### 乘积最大子序列<br>
![微信截图_20191009150822.png](https://i.loli.net/2019/10/10/rkKifus5RlBHcP1.png)

根据题意很容易想出状态方程，dp[i] = max(dp\[i-1]*nums[i],nums[i])。但需要注意nums[i]等于负数时，需要把累成值存在cur_min下，因为如果第二次出现负数，其累乘将会成为最大值。为了避免额外空间，用res来更新当前最大值。另外cur_max记录当前最大值，当出现负数时，cur_max和cur_min交替即可。

In [10]:
class Solution(object):
    def maxProduct(self, nums):
        n = len(nums)
        if not n: return 0
        cur_max = cur_min = res = nums[0]
        for i in range(1,n):
            if nums[i] < 0:
                cur_max, cur_min = cur_min, cur_max
            cur_max = max(cur_max*nums[i],nums[i])
            cur_min = min(cur_min*nums[i],nums[i])
            res = max(cur_max,res)
        return res